<center>

# [Компьютерное зрение](http://rairi.ru/wiki/index.php/%D0%9A%D0%BE%D0%BC%D0%BF%D1%8C%D1%8E%D1%82%D0%B5%D1%80%D0%BD%D0%BE%D0%B5_%D0%B7%D1%80%D0%B5%D0%BD%D0%B8%D0%B5)

## <center> Семинар 13 - 3D детекция с использованием лидарных облаков точек

<a target="_blank" href="https://colab.research.google.com/github/alexmelekhin/cv_course_2023/blob/main/seminars/seminar_13/Seminar_13.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

***

# Установка MMDetection3D

Установка осуществляется согласно инструкции [Get Started](https://github.com/open-mmlab/mmdetection3d/blob/main/docs/en/get_started.md) из оригинального репозитория.

In [ ]:
!pip3 install openmim
!mim install mmengine
!mim install "mmcv>=2.0.0rc4,<2.1.0"
!mim install "mmdet>=3.0.0,<3.1.0"

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection3d.git -b dev-1.x
%cd mmdetection3d
!pip install -e .

In [ ]:
import mmdet3d
print(mmdet3d.__version__)
# Example output: 1.1.0rc0, or an another version.

1.1.0


In [ ]:
!mim download mmdet3d --config pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car --dest .

processing pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MiB 22.0 MB/s eta 0:00:00
Successfully downloaded hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth to /content
Successfully dumped pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py to /content


# VoxelNet

https://arxiv.org/abs/1711.06396

> Accurate detection of objects in 3D point clouds is a central problem in many applications, such as autonomous navigation, housekeeping robots, and augmented/virtual reality. To interface a highly sparse LiDAR point cloud with a region proposal network (RPN), most existing efforts have focused on hand-crafted feature representations, for example, a bird's eye view projection. In this work, we remove the need of manual feature engineering for 3D point clouds and propose VoxelNet, a generic 3D detection network that unifies feature extraction and bounding box prediction into a single stage, end-to-end trainable deep network. Specifically, VoxelNet divides a point cloud into equally spaced 3D voxels and transforms a group of points within each voxel into a unified feature representation through the newly introduced voxel feature encoding (VFE) layer. In this way, the point cloud is encoded as a descriptive volumetric representation, which is then connected to a RPN to generate detections. Experiments on the KITTI car detection benchmark show that VoxelNet outperforms the state-of-the-art LiDAR based 3D detection methods by a large margin. Furthermore, our network learns an effective discriminative representation of objects with various geometries, leading to encouraging results in 3D detection of pedestrians and cyclists, based on only LiDAR.

![voxelnet.png](https://github.com/steph1793/Voxelnet/raw/master/images/pre.png)

# PointPillars

https://github.com/open-mmlab/mmdetection3d/tree/main/configs/pointpillars 

![pointpillars.png](https://user-images.githubusercontent.com/79644370/143885905-aab6ffcf-7727-495e-90ca-edb8dd5e324b.png)

In [ ]:
from mmdet3d.apis import init_model, inference_detector

config_file = 'pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py'
checkpoint_file = 'hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth'
model = init_model(config_file, checkpoint_file)

/content/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:92: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


Loads checkpoint by local backend from path: hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth


In [ ]:
model

VoxelNet(
  (data_preprocessor): Det3DDataPreprocessor(
    (voxel_layer): VoxelizationByGridShape(voxel_size=[0.16, 0.16, 4], grid_shape=[432, 496, 1], point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1], max_num_points=32, max_voxels=(16000, 40000), deterministic=True)
  )
  (backbone): SECOND(
    (blocks): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (5): ReLU(inplace=True)
        (6): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (7): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (8): ReLU(inplace=True)
   

## Pillars Feature Net

https://github.com/open-mmlab/mmdetection3d/blob/1.0/mmdet3d/models/voxel_encoders/pillar_encoder.py#L13



## Backbone + neck - SECOND + FPN

https://github.com/open-mmlab/mmdetection3d/blob/1.0/mmdet3d/models/backbones/second.py#L12

https://github.com/open-mmlab/mmdetection3d/blob/1.0/mmdet3d/models/necks/second_fpn.py#L12

## Head - Anchor3DHead

https://github.com/open-mmlab/mmdetection3d/blob/1.0/mmdet3d/models/dense_heads/anchor3d_head.py#L16

# Инференс

In [ ]:
result, data = inference_detector(model, 'mmdetection3d/demo/data/kitti/000008.bin')

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
data["inputs"].keys()

dict_keys(['points'])

In [ ]:
result

<Det3DDataSample(

    META INFORMATION
    box_mode_3d: <Box3DMode.LIDAR: 0>
    pcd_horizontal_flip: False
    pcd_vertical_flip: False
    pcd_trans: array([0., 0., 0.])
    box_type_3d: <class 'mmdet3d.structures.bbox_3d.lidar_box3d.LiDARInstance3DBoxes'>
    lidar_path: 'mmdetection3d/demo/data/kitti/000008.bin'
    pcd_rotation_angle: 0.0
    transformation_3d_flow: ['R', 'S', 'T']
    flip: False
    pcd_scale_factor: 1.0
    pcd_rotation: tensor([[1., 0., 0.],
                [-0., 1., 0.],
                [0., 0., 1.]])

    DATA FIELDS
    pred_instances_3d: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            labels_3d: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
            bboxes_3d: LiDARInstance3DBoxes(
                    tensor([[ 14.7584,  -1.0538,  -1.5590,   3.7563,   1.6060,   1.5587,  -0.3131],
                        [  6.4378,  -3.8679,  -1.7354,   3.1476,   1.4599,   1.4284,  -0.2997],
                   

In [ ]:
result.pred_instances_3d

<InstanceData(

    META INFORMATION

    DATA FIELDS
    labels_3d: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
    bboxes_3d: LiDARInstance3DBoxes(
            tensor([[ 14.7584,  -1.0538,  -1.5590,   3.7563,   1.6060,   1.5587,  -0.3131],
                [  6.4378,  -3.8679,  -1.7354,   3.1476,   1.4599,   1.4284,  -0.2997],
                [  8.1122,   1.2170,  -1.6340,   3.6663,   1.5731,   1.5915,   2.8162],
                [ 20.1698,  -8.4311,  -1.6691,   2.3816,   1.5176,   1.5694,  -0.3255],
                [ 33.4559,  -7.0358,  -1.3376,   4.2141,   1.7446,   1.6697,   2.8285],
                [ 55.6216, -20.3288,  -1.3771,   4.3705,   1.7360,   1.7068,   2.8503],
                [  3.6383,   2.7381,  -1.6892,   3.7205,   1.5820,   1.5178,  -0.2305],
                [ 25.0405, -10.1565,  -1.6325,   3.7390,   1.6085,   1.4840,  -0.3298],
                [ 28.7253,  -1.5525,  -1.2017,   3.6939,   1.5429,   1.5609,   1.2417],
                [ 40.8709,  -9.7488,  -1.3

### LiDARInstance3DBoxes

https://github.com/open-mmlab/mmdetection3d/blob/main/mmdet3d/structures/bbox_3d/lidar_box3d.py



```
Each row is (x, y, z, x_size, y_size, z_size, yaw)
```



# Визуализация

https://mmdetection3d.readthedocs.io/en/latest/user_guides/visualization.html

! Не работает в Google Colab

In [ ]:
from mmdet3d.visualization import Det3DLocalVisualizer

points = data["inputs"]["points"].numpy()

visualizer = Det3DLocalVisualizer()

# set point cloud in visualizer
visualizer.set_points(points)
bboxes_3d = result.pred_instances_3d.bboxes_3d
visualizer.draw_bboxes_3d(bboxes_3d)
visualizer.show()

05/04 07:27:48 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


# Задание

По примеру кода выше запустите инференс модели CenterPoint. Вы можете использовать любую конфигурацию из предложенных в MMDetection3D.

В качестве входных данных используйте `mmdetection3d/demo/data/nuscenes/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402927647951.pcd.bin`.

- https://github.com/open-mmlab/mmdetection3d/tree/main/configs/centerpoint
- https://arxiv.org/abs/2006.11275


### Вопрос 1

Кратко опишите ключевую идею 2D CenterNet детектора.

**Ответ:** данный детектор переформулирует задачу детекции как задачу keypoint estimation. CenterNet учится предсказывать heatmaps with rendered Gaussian kernels для каждого объекта с заданными коодинатами центра и меткой класса и предсказывать bbox (ака 9 чисел) и confidence, решая задачу регрессии.

Плюс подхода в том, что используются уже придуманные подходы к 3D-детекции, VoxelNet и PointPillars.



### Вопрос 2

Как осуществляется трекинг в CenterPoint?

**Ответ:**
Хорошая цитата из статьи!
```
At test time, the detector produces K heatmaps and
dense class-agnostic regression maps. Each local maxima
(peak) in the heatmaps corresponds to an object, with
confidence proportional to the heatmap value at the peak.
For each detected object, the detector retrieves all regression
values from the regression maps at the corresponding
peak location. Depending on the application domain,
Non-Maxima Suppression (NMS) may be warranted.
```


### Вопрос 3

Кратко опишите две стадии работы метода CenterPoint.

**Ответ:**
1. Centers and 3D boxes: Для каждой точки определяется свой вектор признаков, используя билинейную интерполяцию и CenterNet (3D -> 2D свёртки), потом это всё конкатенируется и прогоняется через MLP.

2. Score and 3D boxes использует MLP-слои выше.

```
We rely on a standard 3D backbone that extracts map-view feature
representation from Lidar point-clouds. Then, a 2D CNN architecture detection head finds object centers and regress to full 3D
bounding boxes using center features. This box prediction is used to extract point features at the 3D centers of each face of
the estimated 3D bounding box, which are passed into MLP to predict an IoU-guided confidence score and box regression
refinement. Best viewed in color
```

In [ ]:
!mim download mmdet3d --config centerpoint_pillar02_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d --dest .

processing centerpoint_pillar02_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MiB 77.7 MB/s eta 0:00:00
Successfully downloaded centerpoint_02pillar_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220811_031844-191a3822.pth to /content
Successfully dumped centerpoint_pillar02_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d.py to /content


In [ ]:
from mmdet3d.apis import init_model, inference_detector

config_file = '/content/centerpoint_pillar02_second_secfpn_head-circlenms_8xb4-cyclic-20e_nus-3d.py'
checkpoint_file = '/content/centerpoint_02pillar_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220811_031844-191a3822.pth'
model = init_model(config_file, checkpoint_file)

Loads checkpoint by local backend from path: /content/centerpoint_02pillar_second_secfpn_circlenms_4x8_cyclic_20e_nus_20220811_031844-191a3822.pth


In [ ]:
model

CenterPoint(
  (data_preprocessor): Det3DDataPreprocessor(
    (voxel_layer): VoxelizationByGridShape(voxel_size=[0.2, 0.2, 8], grid_shape=[512, 512, 1], point_cloud_range=[-51.2, -51.2, -5.0, 51.2, 51.2, 3.0], max_num_points=20, max_voxels=(30000, 40000), deterministic=True)
  )
  (pts_voxel_encoder): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (norm): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (linear): Linear(in_features=11, out_features=64, bias=False)
      )
    )
  )
  (pts_middle_encoder): PointPillarsScatter()
  (pts_backbone): SECOND(
    (blocks): ModuleList(
      (0): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): 

In [ ]:
result, data = inference_detector(model, 'mmdetection3d/demo/data/nuscenes/n015-2018-07-24-11-22-45+0800__LIDAR_TOP__1532402927647951.pcd.bin')

/content/mmdetection3d/mmdet3d/models/task_modules/coders/centerpoint_bbox_coders.py:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.post_center_range = torch.tensor(
/content/mmdetection3d/mmdet3d/models/task_modules/coders/centerpoint_bbox_coders.py:201: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.post_center_range = torch.tensor(


In [ ]:
result.pred_instances_3d

<InstanceData(

    META INFORMATION

    DATA FIELDS
    scores_3d: tensor([0.8391, 0.5372, 0.4811, 0.3700, 0.2843, 0.2700, 0.2440, 0.2256, 0.2125,
                0.1970, 0.1968, 0.1948, 0.1863, 0.1812, 0.1811, 0.1754, 0.1707, 0.1612,
                0.1457, 0.1431, 0.1349, 0.1181, 0.1128, 0.1090, 0.1071, 0.5675, 0.4992,
                0.2580, 0.2041, 0.1980, 0.1913, 0.1831, 0.1810, 0.1443, 0.1401, 0.1389,
                0.1335, 0.1333, 0.1331, 0.1326, 0.1274, 0.1267, 0.1222, 0.1219, 0.1214,
                0.1203, 0.1187, 0.1172, 0.1125, 0.1114, 0.1104, 0.1104, 0.1085, 0.1036,
                0.1032, 0.1031, 0.1029, 0.1016, 0.1007, 0.4826, 0.3969, 0.3348, 0.3063,
                0.1958, 0.1143, 0.1121, 0.7830, 0.7243, 0.7099, 0.7094, 0.6948, 0.6908,
                0.6385, 0.6287, 0.6256, 0.6225, 0.5902, 0.5851, 0.5834, 0.5701, 0.5444,
                0.5308, 0.5295, 0.5216, 0.5005, 0.4948, 0.4342, 0.4326, 0.3933, 0.3599,
                0.3351, 0.3238, 0.2838, 0.2585, 0.2178, 0.2